# top 12 decorators

## 1 — @logger (to get started)✏️

In [ ]:
1 — @logger (to get started)✏️
If you’re new to decorators, you can think of them as functions that take functions as input and extend their functionalities without altering their primary purpose.

Let’s start with a simple decorator that extends a function by logging when it starts and ends executing.

The result of the function being decorated would look like this:

some_function(args)

# ----- some_function: start -----
# some_function executing
# ----- some_function: end -----
To write this decroator, you first have to pick an appropriate name: let’s call it logger.

logger is a function that takes a function as input and returns a function as output. The output function is usually an extended version of the input. In our case, we want the output function to surround the call of the input function with start and end statements.

Since we don’t know what arguments the input function use, we can pass them from the wrapper function using *args and **kwargs. These expressions allow passing an arbitrary number of positional and keyword arguments.

Here’s a simple implementation of the logger decorator:

def logger(function):
    def wrapper(*args, **kwargs):
        print(f"----- {function.__name__}: start -----")
        output = function(*args, **kwargs)
        print(f"----- {function.__name__}: end -----")
        return output
    return wrapper
Now you can apply logger to some_function or any other function for that matter.

decorated_function = logger(some_function)
Python provides a more pythonic syntax for this, it uses the @ symbol.

@logger
def some_function(text):
    print(text)

some_function("first test")
# ----- some_function: start -----
# first test
# ----- some_function: end -----

some_function("second test")
# ----- some_function: start -----
# second test
# ----- some_function: end -----
